# Calculo de la Rentabilidad de Un proyecto

## 1. Import Files 

In [1]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import scipy.stats as stats
import matplotlib.pyplot as plt

## 2. Set up

### 2.1 Cargar los datos

In [ ]:
data_path = 'data/INPUT.xlsx'

try:
    input = {
        'param': pd.read_excel(data_path, sheet_name='Parametros'),
        'recursos_ing': pd.read_excel(data_path, sheet_name='Recursos_ing'),
        'cv': pd.read_excel(data_path, sheet_name='CV'),#costos variables
        'cf': pd.read_excel(data_path, sheet_name='CF')#costos fijos
    }
except FileNotFoundError:
    print(f'No se encontró el archivo {data_path}')
except Exception as e:
    print(f'Error al leer el archivo {data_path}: {e}')

for key, value in input.items():
    print(f"Sheet name: {key}")
    print(value)
    print("\n")

Sheet name: param
      Inversion  Impuestos  Numero_de_iteraciones  Numero_annos  \
0  1.000000e+10        0.3                10000.0          15.0   
1           NaN        NaN                    NaN           NaN   
2           NaN        NaN                    NaN           NaN   

   annos_de_inversion  %_inv_por_anno  
0                 3.0             0.2  
1                 NaN             0.5  
2                 NaN             0.3  


Sheet name: recursos_ing
  Nombre del recurso  precio_P_0  precio_P_25  precio_P_50  precio_P_75  \
0  Tarjetas graficas       15000        17000        20000        25000   
1         Procesador        3500         4000         6000         8000   
2                RAM         450          500          600          800   

   precio_P_100  cantidad_P_0  cantidad_P_25  cantidad_P_50  cantidad_P_75  \
0         30000         50000         750000         100000         200000   
1         15000         12500         187500          25000          

### 2.2 Defenir parametros

In [ ]:
n_iteraciones= input['param']['n_iteraciones'][0]
inversion= input['param']['inversion'][0]
impuestos= input['param']['impuestos'][0]
numero_annos= input['param']['numero_annos'][0]
porcen_inv_annos= input['param']['porcen_inv_annos'][0:numero_annos]
print(f"n_iteraciones: {n_iteraciones}")
print(f"inversion: {inversion}")
print(f"impuestos: {impuestos}")
print(f"numero_annos: {numero_annos}")
print(f"porcen_inv_annos: {porcen_inv_annos}")

## 3. Definir la recta de los valores input

In [3]:


def get_ecuation(input):
    precio_str="precio_P_"
    cantidad_str="cantidad_P_"
    recurso={}
    x = np.array([0, 0.25, 0.5, 0.75, 1.0])
    for key, value in input.iterrows():
        print(f"Recursos de ingreso {key}")
        
        tmp_precio = []
        tmp_cantidad = []

        for j in range(0, 5):
            tmp_precio.append(value[f"{precio_str}{j*25}"])
            tmp_cantidad.append(value[f"{cantidad_str}{j*25}"])

            
        #Sacar la ecuacion de la recta del recurso
        recurso[key] = {}
        recurso[key]['precio'] = np.polyfit(x, tmp_precio, 1)
        recurso[key]['cantidad'] = np.polyfit(x, tmp_cantidad, 1)
        print(f"Linear equation for precio: {recurso[key]['precio']}")
        print(f"Linear equation for cantidad: {recurso[key]['cantidad']}")
    return recurso
recurso=get_ecuation(input['recursos_ing'])

print(recurso)

Recursos de ingreso 0
Linear equation for precio: [15200. 13800.]
Linear equation for cantidad: [140000. 250000.]
Recursos de ingreso 1
Linear equation for precio: [10800.  1900.]
Linear equation for cantidad: [35000. 62500.]
Recursos de ingreso 2
Linear equation for precio: [480. 410.]
Linear equation for cantidad: [ 70000. 125000.]
{0: {'precio': array([15200., 13800.]), 'cantidad': array([140000., 250000.])}, 1: {'precio': array([10800.,  1900.]), 'cantidad': array([35000., 62500.])}, 2: {'precio': array([480., 410.]), 'cantidad': array([ 70000., 125000.])}}


## 4 Declarar las funciones
Declaramos las funciones que se usaran en la simulacion de Montecarlo

### 4.1 Sacar valores aleatorios de los recursos

In [4]:
def generate_random_values(recurso):
    random_values = {}
    for key, value in recurso.items():
        a,b=np.random.uniform(0,1),np.random.uniform(0,1)
        random_values[key] = {"precio": value['precio'][0]*a+value['precio'][1], "cantidad": value['cantidad'][0]*b+value['cantidad'][1]}
    return random_values
print(generate_random_values(recurso))

{0: {'precio': np.float64(21581.014494467083), 'cantidad': np.float64(387411.77263749775)}, 1: {'precio': np.float64(8299.612066649586), 'cantidad': np.float64(93869.34331390221)}, 2: {'precio': np.float64(508.27892772398934), 'cantidad': np.float64(181854.90538148934)}}
